In [ ]:
!pip install transformers
!pip install -U -q PyDrive

In [2]:
import pandas as pd
df=pd.read_csv('/content/tam_3_train.tsv', delimiter='\t') 
df=df[["text", "category"]]
df = df.dropna()
df_dev=pd.read_csv('/content/tam_3_dev.tsv', delimiter='\t') 
df_dev=df_dev[["text", "category"]]
df_dev = df_dev.dropna()
df_test=pd.read_csv('/content/tam_3_test.tsv', delimiter='\t')
df_test = df_test.rename(columns={'text                        ': 'text'}) 
df_test = df_test.dropna()

In [3]:
df

,text,category
0,அர்த்தநாரீஸ்வரர் ஆணுமானவர் பெண்ணுமானவர். இது ஒ...,Transphobic
1,section377 | ஓரினச்சேர்க்கை நல்லதா கெட்டதா மக்...,Non-anti-LGBT+ content
2,இருக்கிற நாலு பேர் சேர்ந்து அந்த பொறம்போக்கு ச...,Transphobic
3,Shanth sha உடலுறவு இனப்பெருக்கத்திற்கு மட்டும...,Non-anti-LGBT+ content
4,ஜீவநதி ப்ரியன் hi,Non-anti-LGBT+ content
...,...,...
2657,நீங்கள் என்னதான் சட்டம் கொண்டுவந்தாலும் இது தவ...,Homophobic
2658,Ayy Ar இதய துடிப்பில் உனது நினைவலைகள். ஊன் உற...,Non-anti-LGBT+ content
2659,இவர்கள் இவ்வளவு பண்ணிவிட்டு எங்களை சமுதாயம் மத...,Non-anti-LGBT+ content
2660,ப்ரோ அவங்களுக்கு.திருநங்கைகள் தான் இவங்க.கட்டப...,Non-anti-LGBT+ content


In [4]:
df['category'] = df['category'].replace('Non-anti-LGBT+ content', 'ally')
df_dev['category'] = df_dev['category'].replace('Non-anti-LGBT+ content', 'ally')
df_test['category'] = df_test['category'].replace('Non-anti-LGBT+ content', 'ally')

In [5]:
#homophobic_data

In [ ]:
!pip install nlpaug
!pip install swifter

In [7]:
df['category'] = df['category'].replace('ally', 0)
df_dev['category'] = df_dev['category'].replace('ally', 0)
df['category'] = df['category'].replace('Homophobic', 1)
df_dev['category'] = df_dev['category'].replace('Homophobic', 1)
df['category'] = df['category'].replace('Transphobic', 2)
df_dev['category'] = df_dev['category'].replace('Transphobic', 2)
df_test['category'] = df_test['category'].replace('ally', 0)
df_test['category'] = df_test['category'].replace('Homophobic', 1)
df_test['category'] = df_test['category'].replace('Transphobic', 2)

In [ ]:
# install simpletransformers
!pip install simpletransformers

# check installed version
!pip freeze | grep simpletransformers
# simpletransformers==0.28.2

In [9]:
import torch
torch.cuda.empty_cache()
torch.cuda.memory_summary(device=None, abbreviated=False)

'|===========================================================================|\n|                  PyTorch CUDA memory summary, device ID 0                 |\n|---------------------------------------------------------------------------|\n|            CUDA OOMs: 0            |        cudaMalloc retries: 0         |\n|===========================================================================|\n|        Metric         | Cur Usage  | Peak Usage | Tot Alloc  | Tot Freed  |\n|---------------------------------------------------------------------------|\n| Allocated memory      |      0 B   |      0 B   |      0 B   |      0 B   |\n|       from large pool |      0 B   |      0 B   |      0 B   |      0 B   |\n|       from small pool |      0 B   |      0 B   |      0 B   |      0 B   |\n|---------------------------------------------------------------------------|\n| Active memory         |      0 B   |      0 B   |      0 B   |      0 B   |\n|       from large pool |      0 B   |      0 B   |

In [10]:
df=df[["text", "category"]]
df = df.rename(columns={'text': 'text', 'category': 'labels'})
df_dev=df_dev[["text", "category"]]
df_dev = df_dev.rename(columns={'text': 'text', 'category': 'labels'})
df_test=df_test[["text", "category"]]
df_test = df_test.rename(columns={'text': 'text', 'category': 'labels'})

In [ ]:
! pip install emoji --upgrade

In [12]:
import emoji
df['text'] = df['text'].apply(lambda x: emoji.demojize(x, delimiters=(" ", " ")))
df_dev['text'] = df_dev['text'].apply(lambda x: emoji.demojize(x, delimiters=(" ", " ")))
df_test['text'] = df_test['text'].apply(lambda x: emoji.demojize(x, delimiters=(" ", " ")))

In [13]:
df['labels'].value_counts()

0    2018
1     485
2     152
Name: labels, dtype: int64

In [14]:
from sklearn.metrics import f1_score, accuracy_score
from simpletransformers.classification import ClassificationModel, ClassificationArgs
import logging

logging.basicConfig(level=logging.INFO)
transformers_logger = logging.getLogger("transformers")
transformers_logger.setLevel(logging.WARNING)
def f1_multiclass(labels, preds):
    return f1_score(labels, preds, average='macro')

# Define hyperparameters to search over
learning_rate_list = [1e-5, 2e-5, 3e-5]


# Loop over all possible combinations of hyperparameters
for learning_rate in learning_rate_list:
        # Define hyperparameter values
    train_args = {"reprocess_input_data": True,
                      "overwrite_output_dir": True,
                      "fp16": False,
                      "num_train_epochs": 3,
                      "max_seq_length": 128,
                      "train_batch_size": 32,
                      "eval_batch_size": 32,
                      "logging_steps": 50,
                      "save_steps": 2000,
                      "learning_rate": learning_rate,
                      "manual_seed": 4}

        # Create a ClassificationModel
    model = ClassificationModel(
            "bert", "bert-base-multilingual-cased",
            num_labels=3,
            args=train_args
        )

        # Train the model and compute the score
    model.train_model(df)
    result, model_outputs, wrong_predictions = model.eval_model(df_dev, f1=f1_multiclass, acc=accuracy_score)
    print(result)

Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model ch

  0%|          | 0/2655 [00:00<?, ?it/s]

Epoch:   0%|          | 0/3 [00:00<?, ?it/s]

Running Epoch 0 of 3:   0%|          | 0/83 [00:00<?, ?it/s]

Running Epoch 1 of 3:   0%|          | 0/83 [00:00<?, ?it/s]

...

Running Epoch 1 of 3:   0%|          | 0/83 [00:00<?, ?it/s]

Running Epoch 2 of 3:   0%|          | 0/83 [00:00<?, ?it/s]

  0%|          | 0/665 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/21 [00:00<?, ?it/s]

{'mcc': 0.6941815307613332, 'f1': 0.7723194308952145, 'acc': 0.8917293233082707, 'eval_loss': 0.3292674198746681}


In [16]:
from sklearn.metrics import f1_score, accuracy_score
from simpletransformers.classification import ClassificationModel, ClassificationArgs
import logging

logging.basicConfig(level=logging.INFO)
transformers_logger = logging.getLogger("transformers")
transformers_logger.setLevel(logging.WARNING)
def f1_multiclass(labels, preds):
    return f1_score(labels, preds, average='macro')

# Define hyperparameters to search over
learning_rate_list = [1e-5, 2e-5, 3e-5]

# Loop over all possible combinations of hyperparameters
for learning_rate in learning_rate_list:
        # Define hyperparameter values
    train_args = {"reprocess_input_data": True,
                      "overwrite_output_dir": True,
                      "fp16": False,
                      "num_train_epochs": 3,
                      "max_seq_length": 128,
                      "train_batch_size": 32,
                      "eval_batch_size": 32,
                      "logging_steps": 50,
                      "save_steps": 2000,
                      "learning_rate": learning_rate,
                      "manual_seed": 4}

        # Create a ClassificationModel
    model = ClassificationModel(
            "xlmroberta", "xlm-roberta-base",
            num_labels=3,
            args=train_args
        )

        # Train the model and compute the score
    model.train_model(df)
    result, model_outputs, wrong_predictions = model.eval_model(df_dev, f1=f1_multiclass, acc=accuracy_score)
    print(result)

Some weights of the model checkpoint at xlm-roberta-base were not used when initializing XLMRobertaForSequenceClassification: ['lm_head.layer_norm.bias', 'lm_head.layer_norm.weight', 'roberta.pooler.dense.bias', 'roberta.pooler.dense.weight', 'lm_head.dense.bias', 'lm_head.bias', 'lm_head.dense.weight', 'lm_head.decoder.weight']
- This IS expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['classifier.out_p

/usr/local/lib/python3.9/dist-packages/simpletransformers/classification/classification_model.py:486: UserWarning: use_multiprocessing automatically disabled as xlmroberta fails when using multiprocessing for feature conversion.
  warnings.warn(


Epoch:   0%|          | 0/3 [00:00<?, ?it/s]

Running Epoch 0 of 3:   0%|          | 0/83 [00:00<?, ?it/s]

Running Epoch 1 of 3:   0%|          | 0/83 [00:00<?, ?it/s]

...

Running Epoch 1 of 3:   0%|          | 0/83 [00:00<?, ?it/s]

Running Epoch 2 of 3:   0%|          | 0/83 [00:00<?, ?it/s]

  0%|          | 0/665 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/21 [00:00<?, ?it/s]

{'mcc': 0.43373018672102104, 'f1': 0.47607215296503885, 'acc': 0.8285714285714286, 'eval_loss': 0.42298479023433866}


In [18]:
from sklearn.metrics import f1_score, accuracy_score
from simpletransformers.classification import ClassificationModel, ClassificationArgs
import logging

logging.basicConfig(level=logging.INFO)
transformers_logger = logging.getLogger("transformers")
transformers_logger.setLevel(logging.WARNING)
def f1_multiclass(labels, preds):
    return f1_score(labels, preds, average='macro')

# Define hyperparameters to search over
learning_rate_list = [1e-5, 2e-5, 3e-5]


# Loop over all possible combinations of hyperparameters
for learning_rate in learning_rate_list:
        # Define hyperparameter values
    train_args = {"reprocess_input_data": True,
                      "overwrite_output_dir": True,
                      "fp16": False,
                      "num_train_epochs": 3,
                      "max_seq_length": 128,
                      "train_batch_size": 32,
                      "eval_batch_size": 32,
                      "logging_steps": 50,
                      "save_steps": 2000,
                      "learning_rate": learning_rate,
                      "manual_seed": 4}

        # Create a ClassificationModel
    model = ClassificationModel(
                "albert", "ai4bharat/indic-bert",
            num_labels=3,
            args=train_args
        )

        # Train the model and compute the score
    model.train_model(df)
    result, model_outputs, wrong_predictions = model.eval_model(df_dev, f1=f1_multiclass, acc=accuracy_score)
    print(result)

Some weights of the model checkpoint at ai4bharat/indic-bert were not used when initializing AlbertForSequenceClassification: ['predictions.LayerNorm.bias', 'predictions.dense.bias', 'predictions.dense.weight', 'sop_classifier.classifier.weight', 'predictions.bias', 'predictions.decoder.weight', 'predictions.decoder.bias', 'sop_classifier.classifier.bias', 'predictions.LayerNorm.weight']
- This IS expected if you are initializing AlbertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing AlbertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of AlbertForSequenceClassification were not initialized from the model checkpoint at ai4bharat/indi

  0%|          | 0/2655 [00:00<?, ?it/s]

Epoch:   0%|          | 0/3 [00:00<?, ?it/s]

Running Epoch 0 of 3:   0%|          | 0/83 [00:00<?, ?it/s]

Running Epoch 1 of 3:   0%|          | 0/83 [00:00<?, ?it/s]

...

Running Epoch 1 of 3:   0%|          | 0/83 [00:00<?, ?it/s]

Running Epoch 2 of 3:   0%|          | 0/83 [00:00<?, ?it/s]

  0%|          | 0/665 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/21 [00:00<?, ?it/s]

{'mcc': 0.4949340332207119, 'f1': 0.5116724285342552, 'acc': 0.8451127819548873, 'eval_loss': 0.42296492698646726}
